In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 51.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import json
import numpy as np
from transformers import Trainer, TrainingArguments
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import torch
from transformers import DistilBertForTokenClassification

# **Query Resolution**

Reading data file

In [ ]:
f = open('/content/drive/MyDrive/Semester 9/Information retrieval/data/quac_canard/token_classification/train_distant_supervision.json')
data = json.load(f)

Convert it to dataframe

In [ ]:
df = pd.DataFrame(data)
df.head()

,prev_questions,cur_question,overlapping_terms,semantic_terms,gold_terms,answer_text_with_window,answer_text,id,bert_ner_overlap
0,What do we know about Cove Reber? 19-year-old ...,How did he get in?,"[new, singer, announced, lead, old, permanent,...",[],"[new, singer, announced, lead, old, permanent,...","calists on demos, the then 19-year-old Cove Re...","Reber had sent in his demo tape,",C_de8ea5cde2934ac2899df443ca40d105_0_q#1,"[[what, do, we, know, about, cove, reber, ?, 1..."
1,What do we know about Cove Reber? 19-year-old ...,What was the first thing they did together wit...,[reber],[],[reber],fans of both eras (Green Era/Reber Era) debati...,Saosin played the Taste of Chaos tour the foll...,C_de8ea5cde2934ac2899df443ca40d105_0_q#2,"[[what, do, we, know, about, cove, reber, ?, 1..."
2,What do we know about Cove Reber? 19-year-old ...,What happened after that?,"[tour, toured, capitol, states, march, united,...",[],"[tour, toured, capitol, states, march, united,...",to Capitol Records in March and toured the Un...,"That summer, they released the Saosin EP.",C_de8ea5cde2934ac2899df443ca40d105_0_q#3,"[[what, do, we, know, about, cove, reber, ?, 1..."
3,What do we know about Cove Reber? 19-year-old ...,What was a single they released?,"[new, toured, tour, band, demo]",[],"[band, new, toured, tour, demo]",demo versions of songs later recorded on their...,"""Bury Your Head""",C_de8ea5cde2934ac2899df443ca40d105_0_q#4,"[[what, do, we, know, about, cove, reber, ?, 1..."
4,What do we know about Cove Reber? 19-year-old ...,Did they tour again?,"[bury, new, single, head, band]",[],"[band, bury, new, single, head]",r first full-length album. A video to their ne...,The band continued touring for the rest of 200...,C_de8ea5cde2934ac2899df443ca40d105_0_q#5,"[[what, do, we, know, about, cove, reber, ?, 1..."


Pre-processing previous conversation history and answer text

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def pre_process(texts):
  stop_words = stopwords.words('english')
  punctuation = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
  lemmatized_texts = []
  wnl = WordNetLemmatizer()
  for i in range(len(texts)):
    temp = []
    rep_text = texts[i].replace('-',' ')
    rep_text = rep_text.replace('/',' ')
    rep_text = rep_text.replace("\\"," ")
    tokenized_text = nltk.word_tokenize(rep_text)
    for w in range(len(tokenized_text)):
      word = tokenized_text[w].lower()
      if word not in punctuation and word not in stop_words: 
        temp.append(wnl.lemmatize(word))
    lemmatized_texts.append(temp)
  return lemmatized_texts

In [ ]:
df['lemmatized prev qns'] = pre_process(df['prev_questions'])

In [ ]:
df['lemmatized answer text with window'] = pre_process(df['answer_text_with_window'])

Finding overlapping terms between previous conversation history and answer text

In [ ]:
bert_inp = []
bert_op = []
for i in range(len(df['lemmatized prev qns'])):
  temp1 = df['lemmatized prev qns'][i] + ['[SEP]'] + nltk.word_tokenize(df['cur_question'][i])
  bert_inp.append(temp1)
  overlap = list(set(df['lemmatized prev qns'][i]) & set(df['lemmatized answer text with window'][i]))
  temp2 = []
  for j in temp1:
    if j in overlap:
      temp2.append('REL')
    else:
      temp2.append('O')
  bert_op.append(temp2)

Using the above data for BERT - token classification

In [ ]:
bert_data = pd.DataFrame(columns = ['bert_inp', 'bert_opt'])
bert_data['bert_inp'] = bert_inp
bert_data['bert_opt'] = bert_op

Extracting texts & tags for BERT

In [ ]:
texts = bert_data['bert_inp'].to_list()
tags = bert_data['bert_opt'].to_list()

Split data for train & test

In [ ]:
import numpy as np
indices = np.arange(0,len(texts))
train_texts, val_texts, train_tags, val_tags, train_indices, val_indices = train_test_split(texts, tags, indices,test_size=.2,random_state = 42)

Map tag to id and vice-versa

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag_id = {tag: id for id, tag in enumerate(unique_tags)}
id_tag = {id: tag for tag, id in tag_id.items()}

Using Distil Bert Tokenizer from the transformers library 

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]


Tokenize train & val sentences

In [ ]:
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Encoding tags - to avoid the problem of mismatch assignment of tag

In [ ]:
import numpy as np
def encode_tags(tags, encodings):
    labels = [[tag_id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

Converting texts & tags into PyTorch tensor

In [ ]:
class QuACDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") 
val_encodings.pop("offset_mapping")
train_dataset = QuACDataset(train_encodings, train_labels)
val_dataset = QuACDataset(val_encodings, val_labels)

In [ ]:
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Semester 9/Information retrieval/results',          
    num_train_epochs=10,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=64,   
    warmup_steps=5,                
    weight_decay=0.01,               
    logging_dir='/content/drive/MyDrive/Semester 9/Information retrieval/logs',            
    logging_steps=5,
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,        
    eval_dataset=val_dataset           
)

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
model_path = "/content/drive/MyDrive/Semester 9/Information retrieval/distilbert-base-cased_10_epochs"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Initializing the model already saved

In [ ]:
model_path = "/content/drive/MyDrive/checkpoint-4500"
model = DistilBertForTokenClassification.from_pretrained(model_path, num_labels=len(unique_tags))

Resolving queries of Train dataset

In [ ]:
test_trainer = Trainer(model) 
train_raw_pred, _, _ = test_trainer.predict(train_dataset)

***** Running Prediction *****
  Num examples = 25230
  Batch size = 8


In [ ]:
import numpy as np
np.save('/content/drive/MyDrive/Information retrieval/models_IR/train_raw_pred_harini_model',train_raw_pred)

In [ ]:
from numpy import load
train_raw_pred = load('/content/drive/MyDrive/Semester 9/Information retrieval/models_IR/train_raw_pred.npy')

In [ ]:
rel = tag_id['REL']
train_resolved_queries = []
train_predictions = []
train_true = []
for i in range(len(train_texts)):
  prev_count = 0
  yhat = tf.convert_to_tensor(train_raw_pred[i], dtype=tf.float32)
  temp = np.argmax(yhat, axis=1)
  y_pred = [1 if x==1 else 0 for x in temp]
  train_predictions.append(y_pred)

  temp2 = [1 if x=='REL' else 0 for x in train_tags[i]]
  temp2.extend([0] * (len(y_pred)-len(temp2)))
  train_true.append(temp2)
  for j in range(len(train_texts[i])):
    if train_texts[i][j] == '[SEP]':
      break
    prev_count+=1
  res_query = train_texts[i][prev_count+1:]
  for j in range(len(train_tags[i])):
    if train_tags[i][j] == '[SEP]':
      break
    elif train_tags[i][j] == 'REL' and train_texts[i][j] not in res_query:
      res_query.append(train_texts[i][j])
  train_resolved_queries.append(res_query)

Resolving queries of validation datasets 

In [ ]:
test_trainer = Trainer(model) 
val_raw_pred, _, _ = test_trainer.predict(val_dataset)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 6308
  Batch size = 8


In [ ]:
import numpy as np
np.save('/content/drive/MyDrive/Information retrieval/models_IR/val_raw_pred_harini_model',val_raw_pred)

In [ ]:
val_raw_pred = np.load('/content/drive/MyDrive/Semester 9/Information retrieval/models_IR/val_raw_pred.npy')

In [ ]:
val_resolved_queries = []
val_predictions = []
val_true = []
for i in range(len(val_texts)):
  yhat = tf.convert_to_tensor(val_raw_pred[i], dtype=tf.float32)
  temp1 = np.argmax(yhat, axis=1)
  y_pred = [1 if x==1 else 0 for x in temp1]
  val_predictions.append(y_pred)

  temp2 = [1 if x=='REL' else 0 for x in val_tags[i]]
  temp2.extend([0] * (len(y_pred)-len(temp2)))
  val_true.append(temp2)

  prev_count = 0
  for j in range(len(val_texts[i])):
    if val_texts[i][j] == '[SEP]':
      break
    prev_count += 1
  y_pred = y_pred[:prev_count]
  res_query = val_texts[i][prev_count+1:]
  for t in range(len(y_pred)):
    if y_pred[t] == tag_id['REL'] and val_texts[i][t] not in res_query:
      res_query.append(val_texts[i][t])
  val_resolved_queries.append(res_query)

# Evaluating query resolution

In [ ]:
from sklearn.metrics import recall_score,precision_score,f1_score,accuracy_score

In [ ]:
def metrics(y_true, y_pred):
    if any(isinstance(s, list) for s in y_true):
        y_true = [item for sublist in y_true for item in sublist]
        y_pred = [item for sublist in y_pred for item in sublist]
    print("Recall:",recall_score(y_true, y_pred,average='macro'))
    print("Precison:",precision_score(y_true, y_pred,average='macro'))
    print("F1 score:",f1_score(y_true, y_pred,average='macro'))

In [ ]:
print("TRAIN METRICS:")
metrics(train_true,train_predictions)

TRAIN METRICS:
Recall: 0.6196262251630926
Precison: 0.5424638647766551
F1 score: 0.5507304515895669


In [ ]:
print("VALIDATION METRICS:")
metrics(val_true,val_predictions)

VALIDATION METRICS:
Recall: 0.5978364884308214
Precison: 0.5420988925991485
F1 score: 0.5509818433801191


# **Multi-turn passage retrieval pipeline**

In [ ]:
D_copy = []
ans_texts = []
for i in train_indices:
  D_copy.append(df['lemmatized answer text with window'][i])
  ans_texts.append(df['answer_text'][i])
for i in val_indices:
  D_copy.append(df['lemmatized answer text with window'][i])
  ans_texts.append(df['answer_text'][i])
queries = train_resolved_queries+val_resolved_queries

In [ ]:
import itertools
corpus = list(itertools.chain(*D_copy))
all_query_terms = list(itertools.chain(*queries))

In [ ]:
import math
import operator

In [ ]:
!pip install translate

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.5 MB/s 
     |████████████████████████████████| 1.2 MB 11.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=9539114ab27df8a5938045e2e0f79f53855897f68bc204fcbc498f8c98816be0
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [ ]:
freq_all_query_terms = dict()
for i in all_query_terms:
  if i not in freq_all_query_terms:
    freq_all_query_terms[i.lower()] = corpus.count(i.lower())

In [ ]:
import pickle
with open("/content/drive/MyDrive/Semester 9/Information retrieval/freq_all_query_terms_bert.text", "rb") as f1:
  freq_all_query_terms = pickle.load(f1)

In [ ]:
import pandas as pd
import time
import sys
import os
import numpy as np
from translate import Translator
translator=Translator(to_lang='en',from_lang='es')
import sklearn.metrics.pairwise
from tqdm import tnrange
from sklearn.metrics import jaccard_score
import scipy
import re

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('bert-base-nli-mean-tokens') 

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_bert-base-nli-mean-tokens/config.json
Model config BertConfig {
  "_name_or_path": "old_models/bert-base-nli-mean-tokens/0_BERT",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_bert-base-nli-mean-tokens/pytorch_model.bin
All model checkpoint weights were used when initializing BertMo

# Initial ranking 

**Ranking using query likelihood and dirichlet smoothing**

In [ ]:
def initial_ranking(D,Q,alpha = 0.0005):
  query_likelihood = dict()
  for k in range(len(D)):
    doc = D[k]
    mu = (len(doc) * (1-alpha))/alpha
    ql = 0
    for j in range(len(Q)):
      if Q[j].lower() not in doc:
        freq_d = 0
      else:
        freq_d = doc.count(Q[j].lower())
      if Q[j].lower() not in freq_all_query_terms:
        freq_corpus = 0
      else:
        freq_corpus = (freq_all_query_terms[Q[j].lower()]/len(corpus))
      temp = ((freq_d) + (mu*freq_corpus))/(len(doc)+mu)
      #print(temp)
      if temp!=0:
        ql += np.log(temp)
    query_likelihood[k] = ql
  
  temp = dict(sorted(query_likelihood.items(), key=operator.itemgetter(1),reverse=True))
  top_20_passages = dict(itertools.islice(temp.items(), 20))
  L1 = dict()
  for t in top_20_passages:
    L1[t] = query_likelihood[t]
  return L1

# Supervised Neural Ranker - BERT

In [ ]:
import itertools
def BERT_ranking(Q,L1,D):
  query_sentence = []
  query_sentence.append(' '.join(Q))
  query_embedding = embedder.encode(query_sentence)
  passages = []
  for j in L1:
    passages.append(' '.join(D[j]))
  corpus_embeddings = embedder.encode(passages)
  distances = scipy.spatial.distance.cdist(query_embedding, corpus_embeddings, "cosine")[0]
  results = zip(L1.keys(), distances)
  Ln = dict()
  for idx, distance in results:
      Ln[idx] = distance
  Ln = dict(sorted(Ln.items(), key=operator.itemgetter(1),reverse=False))
  #print(Ln)
  Ln = dict(itertools.islice(Ln.items(), 20))
  return Ln

# Rank fusion

**Using Reciprocal Rank Fusion (RRF)**

In [ ]:
def rank_fusion(L1,Ln,D,k=60):
  score = dict()
  for i in L1:
    ind_l1 = list(L1.keys()).index(i)
    ind_ln = list(Ln.keys()).index(i)
    score[i] = ((1/(k + ind_l1)) +((1/(k + ind_ln))))
  L = dict(sorted(score.items(), key=operator.itemgetter(1),reverse=True))
  return L

In [ ]:
sample = [0,4,5,7,8,31510,31511,31512]

In [ ]:
y_true = []
y_pred = []
avg_precision = []
recall = []
for i in sample:
  ind = np.where(train_indices==i)[0]
  if len(ind) != 0:
    Q = queries[ind[0]]
    L1 = initial_ranking(D_copy,Q)
    Ln = BERT_ranking(Q,L1,D_copy)
    L = rank_fusion(L1,Ln,D_copy)
    print("Current query:",df['cur_question'][i])
    print("\nConversational history:",df['prev_questions'][i])
    print("\nResolved query:",Q)
    print("\nTop 20 passages:")
    k = 1
    for j in L:
      print(k,":",ans_texts[j])
      k+=1
      print("\n")
    print("-"*1000)
    true, pred,avg_prec,r = evaluate_performance(L,D_copy,ind[0])
    y_true.append(y_true)
    y_pred.append(y_pred)
    avg_precision.append(avg_prec)
    recall.append(r)
  else:
    #print("val")
    ind = np.where(val_indices==i)[0]
    Q = queries[len(train_resolved_queries)+ind[0]]
    L1 = initial_ranking(D_copy,Q)
    Ln = BERT_ranking(Q,L1,D_copy)
    L = rank_fusion(L1,Ln,D_copy)
    print("Current query:",df['cur_question'][i])
    print("\nConversational history:",df['prev_questions'][i])
    print("\nResolved query:",Q)
    print("\nTop 20 passages:")
    k = 1
    for j in L:
      print(k,":",ans_texts[j])
      k+=1
      print("\n")
    print("-"*1000)
    true, pred,avg_prec,r = evaluate_performance(L,D_copy,len(train_resolved_queries)+ind[0])
    y_true.append(y_true)
    y_pred.append(y_pred)
    avg_precision.append(avg_prec)
    recall.append(r)

Current query: How did he get in?

Conversational history: What do we know about Cove Reber? 19-year-old Cove Reber was announced as their new permanent lead singer.

Resolved query: ['How', 'did', 'he', 'get', 'in', '?', 'cove', 'reber', '19', 'year', 'old', 'announced', 'new', 'permanent', 'lead', 'singer']

Top 20 passages:
1 : Reber had sent in his demo tape,


2 : Saosin played the Taste of Chaos tour the following winter. Saosin was signed to Capitol Records in March and toured the United States with the Warped Tour


3 : considered quitting the show due to the pain.


4 : He had eight procedures while he was on Idol, and considered quitting the show due to the pain.


5 : along with Valerie Adams, Veronica Campbell-Brown, Jacques Freitag, Yelena Isinbayeva, Jana Pittman, Dani Samuels, David Storl, and Kirani James


6 : Nevertheless, Freud had decided that "the Stracheys should become members (full) of the Society.


7 : He would continuously alter and adapt the film up until 19

# Evaluating ranked retrieval

In [ ]:
from sklearn.metrics import average_precision_score,recall_score
def evaluate_performance(scores,indexes,train_indices,val_indices,length):
  y_true = [0 for i in range(20)]
  #y_pred = [0 for i in range(20)]
  keys = list(scores.keys())
  for i in indexes:
    ind = np.where(train_indices==i)[0]
    if len(ind)!=0:
      idx = ind[0]
    else:
      ind = np.where(val_indices==i)[0]
      idx = ind[0]

    if idx in keys:  
      y_true[keys.index(idx)] = 1
    
    y_pred = list(scores.values())

  avg_prec = average_precision_score(np.array(y_true),np.array(y_pred))
  #print(avg_prec)
  return y_true,y_pred,avg_prec

In [ ]:
y_true = []
y_pred = []
avg_precision = []
#recall = []
indexes = [i for i in range(6)]
for i in indexes:
  ind = np.where(train_indices==i)[0]
  if len(ind) != 0:
    Q = queries[ind[0]]
    L1 = initial_ranking(D_copy,Q)
    Ln = BERT_ranking(Q,L1,D_copy)
    L = rank_fusion(L1,Ln,D_copy)
    true, pred,avg_prec = evaluate_performance(L,indexes,train_indices,val_indices,len(train_texts))
    y_true.append(true)
    y_pred.append(pred)
    avg_precision.append(avg_prec)
    #recall.append(r)
  else:
    ind = np.where(val_indices==i)[0]
    Q = queries[len(train_resolved_queries)+ind[0]]
    L1 = initial_ranking(D_copy,Q)
    Ln = BERT_ranking(Q,L1,D_copy)
    L = rank_fusion(L1,Ln,D_copy)
    true, pred,avg_prec = evaluate_performance(L,indexes,train_indices,val_indices,len(train_texts))
    y_true.append(true)
    y_pred.append(pred)
    avg_precision.append(avg_prec)
    #recall.append(r)

In [ ]:
print("Mean Average Precision:",np.average(avg_precision))

Mean Average Precision: 0.7938552188552187
